In [ ]:
import numpy as np
import skimage as sk
import skimage.io as skio

In [ ]:
images=['emir.tif', 'harvesters.tif', 'icon.tif', 'lady.tif', 'melons.tif', 'monastery.jpg', 'onion_church.tif', 'sculpture.tif','self_portrait.tif', 'three_generations.tif', 'tobolsk.jpg', 'train.tif']

In [ ]:
# NCC 
# name of the input file

# functions that might be useful for aligning the images include:
# np.roll, np.sum, sk.transform.rescale (for multiscale)
def align_NCC(a, b):
    best =-np.inf
    disp = (0, 0)
    x_a, y_a =a.shape
    x_b, y_b =b.shape
    
    for i in range(-30, 31):
        for j in range(-30, 31):
            shif_b= np.roll(b,shift=i, axis=0)
            shift_b =np.roll(shif_b, shift=j, axis=1)
            x_min = min(x_a, x_b)
            y_min = min(y_a, y_b)
            a_block =a[:x_min, :y_min]
            b_block = shift_b[:x_min, :y_min]
            # score = np.linalg.norm(a - shift_b)
            a_norm=a_block/np.linalg.norm(a_block)
            b_nomr= b_block/np.linalg.norm(b_block)
            score= np.sum(a_norm*b_nomr)
            if score > best:
                best = score
                disp = (-i, -j)
    # print("Displacement:", disp)
    return disp

In [ ]:
# L2 Norm
def align_l2(a, b):
    # a =(a - np.mean(a)) / np.std(a)
    # b =(b - np.mean(b))/ np.std(b)
    best=np.inf
    disp = (0, 0)
    x_a, y_a =a.shape
    x_b, y_b =b.shape
    
    for i in range(-30, 31):
        for j in range(-30, 31):
            shif_b= np.roll(b,shift=i, axis=0)
            shift_b =np.roll(shif_b, shift=j, axis=1)
            x_min = min(x_a, x_b)
            y_min = min(y_a, y_b)
            a_block =a[:x_min, :y_min]
            b_block = shift_b[:x_min, :y_min]
            score = np.linalg.norm(a_block - b_block)
            # a_norm=a_block/np.linalg.norm(a_block)
            # b_nomr= b_block/np.linalg.norm(b_block)
            # score= np.sum(a_norm*b_nomr)
            if score < best:
                best = score
                disp = (-i, -j)
    # print("Displacement", disp)
    return disp

In [ ]:
# Use Scikit image for edge detection
def edge(a):
    a = np.uint8(a * 255)

    return sk.feature.canny(a, 3)


In [ ]:
# Equalized and Canny Edge detection 
def canny(a):
    a_equal= sk.exposure.equalize_hist(a)
    a_canny= sk.feature.canny(a_equal, 2.5)
    return a_canny



In [ ]:
# Phase Cross Corrolation 

def phase(a,b):
    a = sk.exposure.equalize_hist(a)
    b = sk.exposure.equalize_hist(b)

    shift, error, diffphase = sk.registration.phase_cross_correlation(a, b)
    x,y=shift
    x=int(x)
    y=int(y)
    return (-x,-y)

In [ ]:
# Naive L2
images=['emir.tif', 'harvesters.tif', 'icon.tif', 'lady.tif', 'melons.tif', 'monastery.jpg', 'onion_church.tif', 'sculpture.tif','self_portrait.tif', 'three_generations.tif', 'tobolsk.jpg', 'train.tif']
for i in images:
    imname =i
    # read in the image
    im = skio.imread(imname)

    # convert to double (might want to do this later on to save memory)    
    im = sk.img_as_float(im)


    # compute the height of each part (just 1/3 of total)
    height = np.floor(im.shape[0] / 3.0).astype(int)

    # separate color channels
    b = im[:height]
    g = im[height: 2*height]
    r = im[2*height: 3*height]

    ag = align_l2(g, b)
    ar = align_l2(r, b)
    print("Displacement green:",ag)
    print("Displacement red:",ar)

    g_align=np.roll(g, shift=ag, axis=(0,1))
    r_align=np.roll(r, shift=ar, axis=(0,1))

    # create a color image
    im_out = np.dstack([r_align, g_align, b])
    # has to normalize it
    # when saving have to convert data type, unsigned 8 bit int
    # save the image
    # fname = 'churchout.tif'
    im_out = (im_out * 255).astype(np.uint8)

    # display the image
    skio.imshow(im_out)
    skio.show()


In [ ]:
# Naice L2
images=['church.tif','cathedral.jpg']
for i in images:
    imname =i
    # read in the image
    im = skio.imread(imname)

    # convert to double (might want to do this later on to save memory)    
    im = sk.img_as_float(im)


    # compute the height of each part (just 1/3 of total)
    height = np.floor(im.shape[0] / 3.0).astype(int)

    # separate color channels
    b = im[:height]
    g = im[height: 2*height]
    r = im[2*height: 3*height]

    ag = align_l2(g, b)
    ar = align_l2(r, b)
    print("Displacement green:",ag)
    print("Displacement red:",ar)

    g_align=np.roll(g, shift=ag, axis=(0,1))
    r_align=np.roll(r, shift=ar, axis=(0,1))

    # create a color image
    im_out = np.dstack([r_align, g_align, b])
    # has to normalize it
    # when saving have to convert data type, unsigned 8 bit int
    # save the image
    # fname = 'churchout.tif'
    im_out = (im_out * 255).astype(np.uint8)

    # display the image
    skio.imshow(im_out)
    skio.show()


In [ ]:
# Naive NCC
images=['emir.tif', 'harvesters.tif', 'icon.tif', 'lady.tif', 'melons.tif', 'monastery.jpg', 'onion_church.tif', 'sculpture.tif','self_portrait.tif', 'three_generations.tif', 'tobolsk.jpg', 'train.tif']
for i in images:
    imname =i
    # read in the image
    im = skio.imread(imname)

    # convert to double (might want to do this later on to save memory)    
    im = sk.img_as_float(im)


    # compute the height of each part (just 1/3 of total)
    height = np.floor(im.shape[0] / 3.0).astype(int)

    # separate color channels
    b = im[:height]
    g = im[height: 2*height]
    r = im[2*height: 3*height]

    ag = align_NCC(g, b)
    ar = align_NCC(r, b)
    print("Displacement green:",ag)
    print("Displacement red:",ar)

    g_align=np.roll(g, shift=ag, axis=(0,1))
    r_align=np.roll(r, shift=ar, axis=(0,1))

    # create a color image
    im_out = np.dstack([r_align, g_align, b])
    # has to normalize it
    # when saving have to convert data type, unsigned 8 bit int
    # save the image
    # fname = 'churchout.tif'
    im_out = (im_out * 255).astype(np.uint8)

    # display the image
    skio.imshow(im_out)
    skio.show()


In [ ]:
# Naive NCC
images=['church.tif','cathedral.jpg']
for i in images:
    imname =i
    # read in the image
    im = skio.imread(imname)

    # convert to double (might want to do this later on to save memory)    
    im = sk.img_as_float(im)


    # compute the height of each part (just 1/3 of total)
    height = np.floor(im.shape[0] / 3.0).astype(int)

    # separate color channels
    b = im[:height]
    g = im[height: 2*height]
    r = im[2*height: 3*height]

    ag = align_NCC(g, b)
    ar = align_NCC(r, b)
    print("Displacement green:",ag)
    print("Displacement red:",ar)

    g_align=np.roll(g, shift=ag, axis=(0,1))
    r_align=np.roll(r, shift=ar, axis=(0,1))

    # create a color image
    im_out = np.dstack([r_align, g_align, b])
    # has to normalize it
    # when saving have to convert data type, unsigned 8 bit int
    # save the image
    # fname = 'churchout.tif'
    im_out = (im_out * 255).astype(np.uint8)

    # display the image
    skio.imshow(im_out)
    skio.show()


In [ ]:
# Image Pyramid
def image_pyr (a,b):
    # represents the image at multiple scales (usually scaled by a factor of 2) 
    # and the processing is done sequentially starting from the coarsest scale 
    # (smallest image) and going down the pyramid, updating your estimate as you go. 
    # It is very easy to implement by adding recursive calls to your original single-scale 
    # implementation
    # print("original",a.shape)
    x_curr =0
    y_curr=0
    min_size =300
    if a.shape[0] >min_size or a.shape[1]>min_size:
        # print("resize")
        a_resc = sk.transform.rescale(a, 0.5, anti_aliasing=True)
        b_resc = sk.transform.rescale(b, 0.5, anti_aliasing=True)
        x_curr,y_curr=image_pyr(a_resc,b_resc)
        # print("REcieved", (x_curr,y_curr))
        x_curr=x_curr*2
        y_curr=y_curr*2
        shift_a= np.roll(a,shift=(x_curr,y_curr), axis=(0,1))
        x,y= phase(shift_a, b)
        x_curr+=x
        y_curr+=y
        # print("Recursive call",(x_curr,y_curr))

    else:
        # print("A LEN",a.shape)
        x_curr =0
        y_curr=0
        shift_a=a
        x,y= phase(shift_a, b)
        x_curr+=x
        y_curr+=y
    return (x_curr,y_curr)

In [ ]:
# Image Pyramid
images=['cathedral.jpg','church.tif','emir.tif', 'harvesters.tif', 'icon.tif', 'lady.tif', 'melons.tif', 'monastery.jpg', 'onion_church.tif', 'sculpture.tif','self_portrait.tif', 'three_generations.tif', 'tobolsk.jpg', 'train.tif']
for i in images:
    imname =i
    # read in the image
    im = skio.imread(imname)

    # convert to double (might want to do this later on to save memory)    
    im = sk.img_as_float(im)


    # compute the height of each part (just 1/3 of total)
    height = np.floor(im.shape[0] / 3.0).astype(int)

    # separate color channels
    b = im[:height]
    g = im[height: 2*height]
    r = im[2*height: 3*height]

    ag = image_pyr(g, b)
    ar = image_pyr(r, b)
    print("Displacement green:",ag)
    print("Displacement red:",ar)

    g_align=np.roll(g, shift=ag, axis=(0,1))
    r_align=np.roll(r, shift=ar, axis=(0,1))

    # create a color image
    im_out = np.dstack([r_align, g_align, b])
    # has to normalize it
    # when saving have to convert data type, unsigned 8 bit int
    # save the image
    # fname = 'churchout.tif'
    im_out = (im_out * 255).astype(np.uint8)

    # display the image
    skio.imshow(im_out)
    skio.show()

In [ ]:
# Image pyramid with Canny Edge detection and Equalizing 
def image_pyr_canny (a,b):
    # represents the image at multiple scales (usually scaled by a factor of 2) 
    # and the processing is done sequentially starting from the coarsest scale 
    # (smallest image) and going down the pyramid, updating your estimate as you go. 
    # It is very easy to implement by adding recursive calls to your original single-scale 
    # implementation
    a= canny(a)
    b=canny(b)
    # print("original",a.shape)
    x_curr =0
    y_curr=0
    min_size =300
    if a.shape[0] >min_size or a.shape[1]>min_size:
        # print("resize")
        a_resc = sk.transform.rescale(a, 0.5, anti_aliasing=False)
        b_resc = sk.transform.rescale(b, 0.5, anti_aliasing=False)
        x_curr,y_curr=image_pyr_canny(a_resc,b_resc)
        # print("REcieved", (x_curr,y_curr))
        x_curr=x_curr*2
        y_curr=y_curr*2
        shift_a= np.roll(a,shift=(x_curr,y_curr), axis=(0,1))
        x,y= phase(shift_a, b)
        x_curr+=x
        y_curr+=y
        # print("Recursive call",(x_curr,y_curr))

    else:
        # print("A LEN",a.shape)
        x_curr =0
        y_curr=0
        shift_a=a
        x,y= phase(shift_a, b)
        x_curr+=x
        y_curr+=y
    return (x_curr,y_curr)

In [ ]:
# Image Pyramid
images=['cathedral.jpg','church.tif','emir.tif', 'harvesters.tif', 'icon.tif', 'lady.tif', 'melons.tif', 'monastery.jpg', 'onion_church.tif', 'sculpture.tif','self_portrait.tif', 'three_generations.tif', 'tobolsk.jpg', 'train.tif']
for i in images:
    imname =i
    # read in the image
    im = skio.imread(imname)

    # convert to double (might want to do this later on to save memory)    
    im = sk.img_as_float(im)


    # compute the height of each part (just 1/3 of total)
    height = np.floor(im.shape[0] / 3.0).astype(int)

    # separate color channels
    b = im[:height]
    g = im[height: 2*height]
    r = im[2*height: 3*height]

    ag = image_pyr_canny(g, b)
    ar = image_pyr_canny(r, b)
    print("Displacement green:",ag)
    print("Displacement red:",ar)

    g_align=np.roll(g, shift=ag, axis=(0,1))
    r_align=np.roll(r, shift=ar, axis=(0,1))

    # create a color image
    im_out = np.dstack([r_align, g_align, b])
    # has to normalize it
    # when saving have to convert data type, unsigned 8 bit int
    # save the image
    # fname = 'churchout.tif'
    im_out = (im_out * 255).astype(np.uint8)

    # display the image
    skio.imshow(im_out)
    skio.show()

In [ ]:
# Image Pyramid
def image_pyr_l2 (a,b):
    # represents the image at multiple scales (usually scaled by a factor of 2) 
    # and the processing is done sequentially starting from the coarsest scale 
    # (smallest image) and going down the pyramid, updating your estimate as you go. 
    # It is very easy to implement by adding recursive calls to your original single-scale 
    # implementation
    # print("original",a.shape)
    x_curr =0
    y_curr=0
    min_size =300
    if a.shape[0] >min_size or a.shape[1]>min_size:
        # print("resize")
        a_resc = sk.transform.rescale(a, 0.5, anti_aliasing=True)
        b_resc = sk.transform.rescale(b, 0.5, anti_aliasing=True)
        x_curr,y_curr=image_pyr(a_resc,b_resc)
        # print("REcieved", (x_curr,y_curr))
        x_curr=x_curr*2
        y_curr=y_curr*2
        shift_a= np.roll(a,shift=(x_curr,y_curr), axis=(0,1))
        x,y= align_l2(shift_a, b)
        x_curr+=x
        y_curr+=y
        # print("Recursive call",(x_curr,y_curr))

    else:
        # print("A LEN",a.shape)
        x_curr =0
        y_curr=0
        shift_a=a
        x,y= align_l2(shift_a, b)
        x_curr+=x
        y_curr+=y
    return (x_curr,y_curr)

In [ ]:
# Image Pyramid l2
images=['cathedral.jpg','church.tif','emir.tif', 'harvesters.tif', 'icon.tif', 'lady.tif', 'melons.tif', 'monastery.jpg', 'onion_church.tif', 'sculpture.tif','self_portrait.tif', 'three_generations.tif', 'tobolsk.jpg', 'train.tif']
for i in images:
    imname =i
    # read in the image
    im = skio.imread(imname)

    # convert to double (might want to do this later on to save memory)    
    im = sk.img_as_float(im)


    # compute the height of each part (just 1/3 of total)
    height = np.floor(im.shape[0] / 3.0).astype(int)

    # separate color channels
    b = im[:height]
    g = im[height: 2*height]
    r = im[2*height: 3*height]

    ag = image_pyr_l2(g, b)
    ar = image_pyr_l2(r, b)
    print("Displacement green:",ag)
    print("Displacement red:",ar)

    g_align=np.roll(g, shift=ag, axis=(0,1))
    r_align=np.roll(r, shift=ar, axis=(0,1))

    # create a color image
    im_out = np.dstack([r_align, g_align, b])
    # has to normalize it
    # when saving have to convert data type, unsigned 8 bit int
    # save the image
    im_out = (im_out * 255).astype(np.uint8)

    # display the image
    skio.imshow(im_out)
    skio.show()

In [ ]:
def contrast(a):
    p, p1= np.percentile(a, (4, 95))
    a =sk.exposure.rescale_intensity(a, in_range=(p, p1))
    
    return a

In [ ]:
# Fixing the contrast in the outputed images
# Image Pyramid
images=['cathedral.jpg','church.tif','emir.tif', 'harvesters.tif', 'icon.tif', 'lady.tif', 'melons.tif', 'monastery.jpg', 'onion_church.tif', 'sculpture.tif','self_portrait.tif', 'three_generations.tif', 'tobolsk.jpg', 'train.tif']
for i in images:
    imname =i
    # read in the image
    im = skio.imread(imname)

    # convert to double (might want to do this later on to save memory)    
    im = sk.img_as_float(im)


    # compute the height of each part (just 1/3 of total)
    height = np.floor(im.shape[0] / 3.0).astype(int)

    # separate color channels
    b = im[:height]
    g = im[height: 2*height]
    r = im[2*height: 3*height]

    b=contrast(b)
    g=contrast(g)
    r=contrast(r)


    ag = image_pyr_canny(g, b)
    ar = image_pyr_canny(r, b)
    print("Displacement green:",ag)
    print("Displacement red:",ar)

    g_align=np.roll(g, shift=ag, axis=(0,1))
    r_align=np.roll(r, shift=ar, axis=(0,1))

    # create a color image
    im_out = np.dstack([r_align, g_align, b])
    # has to normalize it
    # when saving have to convert data type, unsigned 8 bit int
    # save the image
    # fname = 'churchout.tif'
    im_out = (im_out * 255).astype(np.uint8)

    # display the image
    skio.imshow(im_out)
    skio.show()